# Power Production - a Machine Learning project

The assignment project for Machine Learning and Statistics, GMIT 2020-2021

Lecturer: Dr. Ian McLoughlin

>Author: **Andrzej Kocielski**  
>Github: [andkoc001](https://github.com/andkoc001/)  
>Email: G00376291@gmit.ie, and.koc001@gmail.com


This Notebook should be read in conjunction with the corresponding README.md file at the project [repository](https://github.com/andkoc001/Machine-Learning-and-Statistics-Project.git) at GitHub.

## To do rough plan

`delete me once job done`

- Project intro
- Do research on wind power - write a short paragraph on findings  
- Read about the tools - Docker, Flask, pickle - include some references  
- Analyse the raw data set  
- Pick one - linear regression - ML algorithm and apply to the data set - explain the model  
- Pick another ML algorithm - Neural Networks regression - apply and explain the model
- Perform accuracy analysis of the predictions  
- Write a python script to run the web service (use pickle module?)  
- Write a brief conclusion
- Style the web service

## Introduction

### Project objectives

The objective of the is to develop a web service to make predictions using Machine Learning (ML) paradigm. 

The goal of the project is to produce a model or models that, based on the provided dataset `powerproduction`, and through applying the appropriate ML techniques, predicts power output generated by wind turbine from wind. The power output predictions should be generated in response to wind speed values to be obtained as HTTP requests.

Further details can be found in the [project brief](https://github.com/andkoc001/Machine-Learning-and-Statistics/blob/main/assessment.pdf).


## Wind Power

Wind power (or wind energy) is a general term describing energy generated from wind, where the wind kinetic enrgy is converted into electrical power. Typically, the power is generated in wind turbins.

There is many factors influencing the generated output, but wind speed is a fundamental contributor. Wikipedia article states that the power is proportional to the third power of the wind speed ([Wind power](https://en.wikipedia.org/wiki/Wind_power)):

$$
P = \frac{1}{2} A \rho v^3
$$

where $P$ is the power output, $A$ corresponds to the size of the turbin, $\rho$ is the air density and $v$ is the wind speed.

However, in practical terms, the observed power output follows a more complex pattern ([Wind education - Wind power](https://energyeducation.ca/encyclopedia/Wind_power)).

![https://energyeducation.ca/wiki/images/thumb/1/19/Windpowercurve.png/400px-Windpowercurve.png](https://energyeducation.ca/wiki/images/thumb/1/19/Windpowercurve.png/400px-Windpowercurve.png)

The wind turbin activates on a certain treshold wind speed - cut-in speed. Below that speed the turbin operation is not economically viable. 

The maximum power output is achieved at the rated wind speed - turbin specific. In the zone between the cut-in and rated speeds, the power increases exponencially with the wind speed. 

Behind the rated speed, the produced output remains approximately flat (or may decline gently), until it reaches the cut-off speed. At such speed, the turbine is shut down in order to prevent them from taking damege. 

It is also worth noting that the above power curve is only a crude approximation of the observed amount of energy produced in reality. 

## Setting the environment

This notebook documents the project development. Please also refer to README.md file in this repository for general information.

### Importing required libraries

For this project the following libraries will be used:

1. Numpy - for numerical operations
2. Pandas - for DataFrame data type
3. Scikit-learn (aka sklearn) - for ML algorithms
4. Matplotlib - for plotting and visualisation
5. Seaborn - for plotting and visualisation

In [ ]:
# ignore deprecated warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns

### Loading the data set from a file

The data set is loaded from the file powerproduction.txt (in the repository). It is stored as a DataFrame and assigned under the name `df_orig`.

In [ ]:
df_orig = pd.read_csv(r"powerproduction.txt")

### A glance into the data set

The dataset we are going to work on this project is `powerproduction`. Let us take a sneak peek as to how this dataset looks like. We will attempt to evaluate its size, basic statistical properties, distributions, etc. as well as produce some plots for a better understanding of its properties.  

In [ ]:
df_orig

In [ ]:
df_orig.describe()

In [ ]:
sns.relplot(data=df_orig, x="speed", y="power", s=10, palette="pastel", height=5, aspect=3)

### First thoughts

From the above dataset description and data points plot, the following conclusions can be drawn.

The data set consist of 500 observations (rows). Each observation consists of 2 attributes (columns): wind speed (`speed`) and corresponding power output (`power`). The units of the values are not explicitly given.

The wind speed values varies from 0 to 25 and are shown in descending order. The power output values varies from 0 to 113.556. There are 49 instances where the power output equals zero.

From the plot one can observe three distinct areas with different behaviour of the data points. I will refer to them as zones a), b), c) and d).

a) With wind speed from 0 to about 8, the power output is near to none. 
b) There is a strong almost linear correlation the wind speed and the power output in wind speed range from about 9 to about 17.
c) When wind speed exceeds approximately 17, its effect on the power output is close to constant, and the wind turbines work with a high performance.
d) Above certain wind speed level, approximately 24, the power output abruptly ceased.

Such a behavior can be, perhaps, explained by the fact that at low wind speed (zone a))the wind turbines do not generate any power (or it is not economically justified). On the other hand, in the zone d) where the winds are very strong, the power output is not produced, as if the turbines are shut off for safety reasons.

Occasionally, there is a number of observations where the power output is zero, even though the wind speeds are in zones allowing for producing the power. These are assumed to be data anomaly (e.g. due to incorrect reading). In total, there is 49 observations with power output equal to zero. 

In [ ]:
# There is about 10% of the observation with zero power output 
df_orig[df_orig.power == 0].count()

In [ ]:
# Histogram of power outputs - frequency of occurance - 'zero' values seem to distort the plot
plt.figure(figsize=(18,4))
sns.distplot(df_orig.power, bins=100, kde=False)

The analysis of the wind speed distribution shows that the wind speed appears to uniformly distribute, with no particular wind speed dominating.

In [ ]:
# what wind speeds dominate - it appears to be more or less uniformely distributed
plt.figure(figsize=(18,2))
sns.distplot(df_orig.speed, bins=100, kde=False)

### Clean the dataset

The data set seems to be distorted with the 'zero' power outputs. Let's remove these observations from the dataset.

The cleaned dataset is assigned to `df` object.

In [ ]:
# clean the dataset by removing all observations where the power output is zero

df = df_orig[df_orig['power'] !=0]
df

## Linear regression

In [ ]:
# Linear regression is an underfitting approximation
plt.figure(figsize=(18,6))
sns.regplot(data=df, x="speed", y="power", scatter_kws={'s':1})

In [ ]:
# Polynomial regression for cleaned dataset

a_plot = sns.lmplot(data=df, x="speed", y="power", order=9, height=5, aspect=3, scatter_kws={'s':1})

a_plot.set(xlim=(0, 25))
a_plot.set(ylim=(0, 120))

plt.show()

The above polynomial appears to closely follow the pattern of the data points in the domain (wind speed in range 0-25).

Let's now apply the Numpy's `polyfit()` to get the value of the coefficients that minimise the squared order and plot the polynomial function.

In [ ]:
plt.rcParams['figure.figsize'] = [18, 6]

In [ ]:
coeff = np.polyfit(df['speed'], df['power'], 9)
#coeff

x = np.linspace(0, 24.5, 101)

yp = np.poly1d(coeff)
xp = plt.plot(x, yp(x))

plt.xlim(0,25)
plt.ylim(0,120)


plt.rcParams['figure.figsize'] = [18, 6]
plt.show()

print("y = ")
print(yp)

## Refs 
---

#### Pickle

- [001] [Python documentation - Pickle module](https://docs.python.org/3/library/pickle.html)
- [002] [Datacamp - Pickle tutorial](https://www.datacamp.com/community/tutorials/pickle-python-tutorial)
- [003] [Stack Overflow - How to read pickle file](https://stackoverflow.com/a/35068080)
- [004] [YouTube - From Jupyter to pickle](https://youtu.be/3-KOE15utuQ)

#### Docker


#### Flask

#### Wind Power

- [001] [Wikipedia - Wind power](https://en.wikipedia.org/wiki/Wind_power)
- [002] [Wind power - How to calculate power output of wind](https://www.windpowerengineering.com/calculate-wind-power-output)
- [003] [Energy education - Wind power](https://energyeducation.ca/encyclopedia/Wind_power)
- [004] [Wind works - wind turbine rating](http://www.wind-works.org/cms/index.php?id=85&tx_ttnews%5Btt_news%5D=2572&cHash=b74148dbab1938159937a994adf1f05b)
